<a href="https://colab.research.google.com/github/hyeonji0401/CUDA_practice/blob/main/cudaMalloc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3.1 CUDA 프로그램의 구조 및 흐름
호스트 코드가 반드시 필요한 이유

> CPU가 운영체제와 같은 컴퓨터 시스템의 기본 연산 장치이며, GPU와 같은 다른 연산 장치를 사용하기 위해서는 호스트 코드에서 커널을 호출해야 하기 때문임

* CPU와 GPU는 서로 독립되 장치로 사용하는 메모리 영역이 다름
* GPU는 디바이스 메모리를 사용하지만 모든 데이터는 기본적으로 호스트 메모리에 저장되어있음

=> GPU를 이용해서 데이터를 처리하기 위해서는 호스트 메모리에 있는 데이터를 디바이스 메모리로 복사해주어야함

<CUDA 프로그램 흐름>
1. 호스트 -> 디바이스 데이터 복사(GPU에서 처리할 데이터)
2. GPU연산
3. 디바이스 -> 호스트 데이터 복사(연산 결과 데이터)





# 3.2 CUDA 기초 메모리 API

디바이스 메모리 공간 할당 및 초기화
> 디바이스 메모리로 데이터를 복사하기 위해서는 메모리에 사용할 공간을 할당받아야함

> 이는 C언어 malloc()함수와 동일함

> 차이점은 C언어는 호스트 공간의 메모리를 할당하는 것이고 cudaMalloc()은 디바이스 메모리 공간을 할당받는 것임

**디바이스 메모리 할당**
* cudaMalloc()

함수 원형


```
cudaError_t cudaMalloc(void **ptr, size_t size)
```
1. void **ptr
- 디바이스 메모리 공간의 시작 주소를 담을 포인터 변수의 주소

2. size_t size
- 할당할 공간의 크기(byte 단위)






In [ ]:
%%cuda

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

int main(void)
{
    int *dDataPtr;
    cudaMalloc(&dDataPtr, sizeof(int)*32);
}


위의 코드는 디바이스 메모리 공간에 int형 데이터 32개를 담을 공간을 할당하는 예시임

위에서 설명했던 것과 같이 &dDatePtr가 할당된 메모리 공간의 시작주소가 저장되는 포인터 변수가 되는 것

* 주의할 점 : dDatePtr이 가리키는 주소는 디바이스 메모리 주소이므로 호스트 코드에서 직접 접근할 수 없다

그래서 만약


In [ ]:
%%cuda

#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>

int main(void)
{
    int *dDataPtr;
    cudaMalloc(&dDataPtr, sizeof(int)*32);
    printf("%d", dDataPtr[0]);
    cudaFree(dDataPtr);
    return 0;
}


이와 같이 데이터에 접근하면 실행 오류가 발생함

**printf()는 CPU에서 작동하는 코드이기 때문임**



> CUDA 프로그램에서는 호스트 메모리 영역을 사용한느 변수와 디바이스 메모리 영역을 사용하는 변수를 구분하기 위해 일반적으로 디바이스 메모리 영역을 사용하는 변수의 이름 앞에 디바이스를 뜻하는 d를 붙여줌

* cudaFree()는 할당된 메모리를 해제하는 함수